# Persönliche Dokumentation für das Tool
## Anforderungen
- CSV Exportdatei aus mLife entgegen nehmen ("gesamte_akte.csv")
- Alle (oder zumindest für RedCap erforderliche) Daten in ein Dataframe parsen
- Darstellung und Konfiguration des Exports mit Streamlit
- Export als CSV, welche in RedCap importiert werden kann

## Übersicht Datenfluss
CSV-Import 
-> Parsing 
-> Dataframe für jede Kategorie 
-> Auswahl in Streamlit & Aggregation
-> RedCap-DB optimiertes DataFrame
-> CSV-Export

## Import
- aus mLife wird eine CSV-Datei der gesamten_akte exportiert; der Delimiter kann zwischen ";" und "|" gewählt werden
### Parsing der Daten in den State (`StateProvider`)
- `state_provider.state_provider_class StateProvider` initiiert sich selbst und exportiert sich als `state_provider`
    - `state_provider` verwaltet und liefert App-weit alle Daten in Form der Pydantic-Klasse `schemas.app_state_schema.app_state AppState`
        - `ParsedData`, `Views` und `UiState`
- `state_provider.parse_data_to_state()` nimmt die importierte CSV-Datei entgegen
    - übergibt die Datei an den Parser `services.data_parser DataParser`, wo durch den Aufruf `DataParser(file, delimiter)` die Parser-Klasser initiiert wird
    - StateProvider ruft die einzelnen Parser-Methoden auf, die wiederum Dataframes für jede Kategorie an Daten (bspw. Laborwerte, Vitalwerte, NIRS) liefern
    - StateProvider erzeugt aus allen generierten DataFrames eine Instanz der Pydantic-Klasse `schemas.app_state_schemas.app_state ParsedData`
    - StateProvider legt diese `ParsedData`-Instanz im `AppState.parsed_data`ab
### Verarbeitung im `DataParser`
- Bei Initiierung lädt der DataParser erstmal nur das File
- wenn das erste mal eine Parsing-Funktion aufgerufen wird, wird die Datei mit `DataParser._split_blocks()` und `DataParser._clean_csv()` bereinigt und als Objekt mit einzelnen Datenblöcken gespeichert nach dem Schema in `DataParser.header`
- `DataParser`enthält dann verschiedene Parser-Methoden, die aus diesem `blocks`-Objekt Strukturierte Pandas DataFrames anhand der Pydantic-Schemata aus `schemas.parse_schemas` erzeugt